In [3]:
from graphframes import *
from functools import reduce
from pyspark import *
from pyspark.sql import *
from operator import add
import shortuuid

In [3]:
#with open("com-orkut.top5000.cmty.txt", 'r') as file:
with open("email-enron.txt", 'r') as file:
    lines = file.read().split("\n")

print("lines")
#print(lines)
vertices = []
existing_vertices = set()
edges = []
hyperedges = {}
counter=0
for l in lines:
    if l == "":
        continue
    counter+=1
    all_line_vertices = l.split(" ")
    hypid = "he" + str(counter)
    try:
        all_line_vertices.remove("")
    except:
        pass
    hyperedges[hypid] = all_line_vertices
    for v in all_line_vertices:
        if not v in existing_vertices:
            vertices.append((v,"vertex"))
            existing_vertices.add(v)
        if not hypid in existing_vertices:
            vertices.append((hypid,"hyperedge"))
            existing_vertices.add(hypid)
        edges.append((v,hypid))
        edges.append((hypid,v))

vertex_metadata = ['id','type']
edges_metadata = ['src','dst']
#print(vertices)
#print(edges)

vertices = spark.createDataFrame(vertices,vertex_metadata)
edges = spark.createDataFrame(edges,edges_metadata)



print("EDGESRCDEST")
print(hyperedges)
hyp_deg = spark.sparkContext.broadcast(hyperedges)
g = GraphFrame(vertices, edges)## Take a look at the DataFrames

lines
EDGESRCDEST
{'he1': ['0', '3'], 'he2': ['0', '112', '123'], 'he3': ['0', '46'], 'he4': ['0', '46'], 'he5': ['0', '36'], 'he6': ['0', '112'], 'he7': ['0', '36'], 'he8': ['0', '60'], 'he9': ['0', '36'], 'he10': ['0', '102'], 'he11': ['0', '102'], 'he12': ['0', '123'], 'he13': ['0', '117'], 'he14': ['0', '46'], 'he15': ['0', '46'], 'he16': ['0', '36'], 'he17': ['0', '26', '117'], 'he18': ['0', '26', '57', '60', '117', '127'], 'he19': ['0', '112'], 'he20': ['0', '3'], 'he21': ['0', '60'], 'he22': ['0', '60'], 'he23': ['0', '46'], 'he24': ['0', '36', '46'], 'he25': ['0', '60'], 'he26': ['0', '36', '60'], 'he27': ['0', '46'], 'he28': ['0', '66'], 'he29': ['0', '112'], 'he30': ['0', '112', '123'], 'he31': ['0', '26'], 'he32': ['0', '26', '36', '46', '60', '92'], 'he33': ['0', '58'], 'he34': ['0', '112'], 'he35': ['0', '26', '36', '46', '117', '127'], 'he36': ['0', '115'], 'he37': ['0', '46'], 'he38': ['0', '93'], 'he39': ['0', '93'], 'he40': ['0', '93'], 'he41': ['0', '26', '36', '46', 

In [5]:
from pyspark.sql.types import Row
import uuid
from pyspark.sql.functions import monotonically_increasing_id
from uuid import UUID

def f(x, uuid):
    d = {}
    elements = x.value.split(" ")
    for i in range(len(elements)):
        yield elements[i],uuid
        yield uuid, elements[i]
    #for i in range(len(x)):
    #    d["abc"] = x[i]
    #return d

def make_edge_df(edge):
    d = {}
    d["src"] = edge[0]
    d["dst"] = edge[1]
    return d

def make_vertex_df(edge):
    yield edge.src,"vertex"
    yield edge.dst, "vertex"
    
        
def hyp_df(x, delimit):
    hyper_return =  x.value.split(delimit)
    try:
        hyper_return.remove("")
    except:
        pass
    return (hyper_return),1

def parse_hyp(x):
    hypid = "hyp" + str(x[len(x)-1])
    for i in range(len(x[0])):
        yield x[0][i],hypid
        yield hypid,x[0][i]

def parse_vert(x):
    hypid = "hyp" + str(x[len(x)-1])
    yield hypid,"hyperedge"
    for i in range(len(x[0])):
        yield x[0][i],"vertex"

def broadcast_hyp(x):
    hypid = "hyp" + str(x[len(x)-1])
    ar = []
    for i in range(len(x[0])):
        ar.append(x[0][i])
    
    return hypid,ar

hyp_lines = spark.read.text("com-orkut.top5000.cmty.txt").rdd.map(lambda x: hyp_df(x, "\t")).toDF()
hyp_lines.show()
hyp_lines = hyp_lines.withColumn("id", monotonically_increasing_id())
hyp_lines.show()

df_edge = hyp_lines.rdd.flatMap(lambda x: parse_hyp(x)).toDF(["src","dst"])
df_edge.show()
df_vert = hyp_lines.rdd.flatMap(lambda x: parse_vert(x)).toDF(["id","type"]).dropDuplicates()
df_hyp = hyp_lines.rdd.map(lambda x: broadcast_hyp(x))
print("df edge",df_edge.show())
print("df vert",df_vert.show())
#print("df hyp", df_hyp.collectAsMap())
new_g = GraphFrame(df_vert, df_edge)
new_hyperedges = df_hyp.collectAsMap()

#hyp_lines.show()
    

+--------------------+---+
|                  _1| _2|
+--------------------+---+
|[2, 5, 6, 37, 38,...|  1|
|[2, 2838, 3406, 3...|  1|
|[4389, 8407, 9492...|  1|
|[6, 18, 19, 21, 2...|  1|
|[8, 26646, 26648,...|  1|
|[9, 39, 42, 65, 8...|  1|
|[960274, 960277, ...|  1|
|[14, 40, 102, 276...|  1|
|[17, 1379, 1389, ...|  1|
|[18, 5840, 5843, ...|  1|
|[18, 722, 725, 72...|  1|
|[20, 55, 61, 181,...|  1|
|[20, 21, 96, 200,...|  1|
|[21, 200, 2312, 2...|  1|
|[22, 669, 1244, 1...|  1|
|[24, 738, 747, 75...|  1|
|[24, 595, 733, 73...|  1|
|[25, 332, 399, 42...|  1|
|[25, 2068, 4962, ...|  1|
|[25, 5184, 5187, ...|  1|
+--------------------+---+
only showing top 20 rows

+--------------------+---+---+
|                  _1| _2| id|
+--------------------+---+---+
|[2, 5, 6, 37, 38,...|  1|  0|
|[2, 2838, 3406, 3...|  1|  1|
|[4389, 8407, 9492...|  1|  2|
|[6, 18, 19, 21, 2...|  1|  3|
|[8, 26646, 26648,...|  1|  4|
|[9, 39, 42, 65, 8...|  1|  5|
|[960274, 960277, ...|  1|  6|
|[14, 40, 102, 2

In [9]:
df_vert.filter('id=="2"').show()

+---+------+
| id|  type|
+---+------+
|  2|vertex|
+---+------+



In [ ]:
df_vertices.filter('id == "56d21a1f-4342-4d22-a486-3ae4960c9890"').show()

In [ ]:
vertex_metadata = ['id','type']
edges_metadata = ['src','dst']
vertices = []
existing_vertices = set()
vex = []
hyperedges = {}
counter=0
existing_vertices = set()
with open("com-orkut.top5000.cmty.txt", 'r') as file:
    lines = file.read().split("\n")

for l in lines:
    if l == "":
        continue
    counter+=1
    all_line_vertices = l.split("\t")
    hypid = "he" + str(counter)
    try:
        all_line_vertices.remove("")
    except:
        pass
    hyperedges[hypid] = all_line_vertices
    for v in all_line_vertices:
        if not v in existing_vertices:
            try:
                frame_vertices = frame_vertices.union(spark.createDataFrame([(v,"vertex")],vertex_metadata))
            except:
                frame_vertices = spark.createDataFrame([(v,"vertex")],vertex_metadata)
            vex.append((v,"vertex"))
            existing_vertices.add(v)
        if not hypid in existing_vertices:
            try:
                frame_vertices = frame_vertices.union(spark.createDataFrame([(hypid,"hyperedge")],vertex_metadata))
            except:
                frame_vertices = spark.createDataFrame([(hypid,"hyperedge")],vertex_metadata)
            vex.append((hypid,"hyperedge"))
            existing_vertices.add(hypid)
        if "ev" in locals() or "ev" in globals():
            ev = ev.union(spark.createDataFrame([(v,hypid)],edges_metadata))
            ev = ev.union(spark.createDataFrame([(hypid,v)],edges_metadata))
        else:
            ev = spark.createDataFrame([(v,hypid)],edges_metadata)
            ev = evg.union(spark.createDataFrame([(hypid,v)],edges_metadata))
#         edges.append((v,hypid))
#         edges.append((hypid,v))

edges.show()
frame_vertices.show()
print(vex)

In [8]:
new_g.vertices.show()
new_g.edges.show()

+-------+------+
|     id|  type|
+-------+------+
|    875|vertex|
|  70395|vertex|
|  73242|vertex|
| 287634|vertex|
| 764572|vertex|
|1968319|vertex|
|2368106|vertex|
|2408366|vertex|
|  20163|vertex|
|  66913|vertex|
| 654675|vertex|
|  93069|vertex|
| 151760|vertex|
|2037144|vertex|
| 142709|vertex|
| 288719|vertex|
| 309770|vertex|
| 164248|vertex|
|  27637|vertex|
| 214165|vertex|
+-------+------+
only showing top 20 rows

+----+----+
| src| dst|
+----+----+
|   2|hyp0|
|hyp0|   2|
|   5|hyp0|
|hyp0|   5|
|   6|hyp0|
|hyp0|   6|
|  37|hyp0|
|hyp0|  37|
|  38|hyp0|
|hyp0|  38|
|  41|hyp0|
|hyp0|  41|
|  44|hyp0|
|hyp0|  44|
|  47|hyp0|
|hyp0|  47|
|  53|hyp0|
|hyp0|  53|
|  56|hyp0|
|hyp0|  56|
+----+----+
only showing top 20 rows



In [10]:
g.edges.show()
g.vertices.show()

+---+---+
|src|dst|
+---+---+
|  0|he1|
|he1|  0|
|  3|he1|
|he1|  3|
|  0|he2|
|he2|  0|
|112|he2|
|he2|112|
|123|he2|
|he2|123|
|  0|he3|
|he3|  0|
| 46|he3|
|he3| 46|
|  0|he4|
|he4|  0|
| 46|he4|
|he4| 46|
|  0|he5|
|he5|  0|
+---+---+
only showing top 20 rows

+----+---------+
|  id|     type|
+----+---------+
|   0|   vertex|
| he1|hyperedge|
|   3|   vertex|
| he2|hyperedge|
| 112|   vertex|
| 123|   vertex|
| he3|hyperedge|
|  46|   vertex|
| he4|hyperedge|
| he5|hyperedge|
|  36|   vertex|
| he6|hyperedge|
| he7|hyperedge|
| he8|hyperedge|
|  60|   vertex|
| he9|hyperedge|
|he10|hyperedge|
| 102|   vertex|
|he11|hyperedge|
|he12|hyperedge|
+----+---------+
only showing top 20 rows



In [10]:

def initialize_page_rank(vertex, count):
    return 1/count

def parseNeighbors(edge):
    dest_id = edge.dst
    if edge.dst in new_hyperedges:
        return edge.src, new_hyperedges[edge.dst]
    return edge.src,[]

In [11]:
def getContributions(ur):
    len_urls = len(ur[1][0])
    for site in sum(ur[1][0],[]):
        yield site, ur[1][1]/(len_urls-1)
    

In [14]:
def parallel_pagerank(iterations, graph):
    all_vertices =graph.vertices.filter('type=="vertex"')
    num_vertices = all_vertices.count()
    all_edges = graph.edges
    all_vertices.show()
    all_edges.show()
    ranks = all_vertices.rdd.map(lambda x:(x.id,initialize_page_rank(x, num_vertices))) #
    hyperedge_links = all_edges.rdd.map(lambda el: parseNeighbors(el)).groupByKey().cache() # vertex hyp1 hyp2 hyp3
    #for i in range(iterations):
    print("hylinks",hyperedge_links.take(5))
    result_links = hyperedge_links.join(ranks).flatMap(lambda ur:getContributions(ur))
    print("reslinks",result_links.take(5))
    ranks = result_links.reduceByKey(add).mapValues(lambda rank: rank * 0.85 + 0.15)
    print(ranks.collect())

In [15]:
parallel_pagerank(5,new_g)

+-------+------+
|     id|  type|
+-------+------+
|    875|vertex|
|  70395|vertex|
|  73242|vertex|
| 287634|vertex|
| 764572|vertex|
|1968319|vertex|
|2368106|vertex|
|2408366|vertex|
|  20163|vertex|
|  66913|vertex|
| 654675|vertex|
|  93069|vertex|
| 151760|vertex|
|2037144|vertex|
| 142709|vertex|
| 288719|vertex|
| 309770|vertex|
| 164248|vertex|
|  27637|vertex|
| 214165|vertex|
+-------+------+
only showing top 20 rows

+----+----+
| src| dst|
+----+----+
|   2|hyp0|
|hyp0|   2|
|   5|hyp0|
|hyp0|   5|
|   6|hyp0|
|hyp0|   6|
|  37|hyp0|
|hyp0|  37|
|  38|hyp0|
|hyp0|  38|
|  41|hyp0|
|hyp0|  41|
|  44|hyp0|
|hyp0|  44|
|  47|hyp0|
|hyp0|  47|
|  53|hyp0|
|hyp0|  53|
|  56|hyp0|
|hyp0|  56|
+----+----+
only showing top 20 rows

hylinks [('1045', <pyspark.resultiterable.ResultIterable object at 0x7f0028d845d0>), ('1071', <pyspark.resultiterable.ResultIterable object at 0x7f0028d84510>), ('3061', <pyspark.resultiterable.ResultIterable object at 0x7f0028d7e310>), ('4641', <pyspa

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.runJob.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 65.0 failed 4 times, most recent failure: Lost task 0.3 in stage 65.0 (TID 146, ip-172-31-23-232.ec2.internal, executor 6): ExecutorLostFailure (executor 6 exited caused by one of the running tasks) Reason: Container killed by YARN for exceeding memory limits.  5.2 GB of 5.0 GB physical memory used. Consider boosting spark.yarn.executor.memoryOverhead or disabling yarn.nodemanager.vmem-check-enabled because of YARN-4714.
Driver stacktrace:
	at org.apache.spark.scheduler.DAGScheduler.org$apache$spark$scheduler$DAGScheduler$$failJobAndIndependentStages(DAGScheduler.scala:2080)
	at org.apache.spark.scheduler.DAGScheduler$$anonfun$abortStage$1.apply(DAGScheduler.scala:2068)
	at org.apache.spark.scheduler.DAGScheduler$$anonfun$abortStage$1.apply(DAGScheduler.scala:2067)
	at scala.collection.mutable.ResizableArray$

In [ ]:
g.edges.filter('src == "1"').show()

In [ ]:
def get_outgoing(node_id, graph):
    df2 = g.edges.filter('src=="'+node_id+'"')
    df2.show()
    df1 = graph.edges
    df3 = df1.alias("df1").join(broadcast(df2.alias("df2")), col("df2.dst") == col("df1.src")).select(df1["dst"])
    df3.distinct().show() 

def get_incoming(node_id, graph):
    df2 = g.edges.filter('dst=="'+node_id+'"')
    df2.show()
    df1 = graph.edges
    df3 = df1.alias("df1").join(broadcast(df2.alias("df2")), col("df2.src") == col("df1.dst")).select(df1["src"])
    df3.distinct().show() 